In [26]:
import os
import json
import pandas as pd
import time
import ast
from pydantic import BaseModel
from pydantic_core import from_json
import glob
from typing import List, Optional
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_together import ChatTogether
import re
import unicodedata
from pathlib import Path

In [27]:
import session_info
session_info.show

<function session_info.main.show(na=True, os=True, cpu=False, jupyter=None, dependencies=None, std_lib=False, private=False, write_req_file=False, req_file_name=None, html=None, excludes=['builtins', 'stdlib_list'])>

In [53]:
TOGETHER_API_KEY= "d979400b673cec622ddde38b0307fc18197e34bf957a91f1b1d58ba647012461"
model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"

In [29]:
class NER(BaseModel):
    personne: Optional[list] = None
    lieu: Optional[list] = None
    organisation: Optional[list] = None

In [30]:
def parse_llm_response(response, basemodel_class = NER):
  try:
    result = basemodel_class.model_validate(from_json(response.content, allow_partial = True))
    category_entity = []
    for entity in result:
      if entity[1] != None:
        category = entity[0]
        entity_text_list = entity[1]

        for ent in entity_text_list:
          category_entity.append((ent, category))

    return category_entity

  except:
    return []

In [31]:
question = "Extrais les entités nommées pertinentes de la phrase suivante."

In [ ]:
personality = "Tu es historien(ne) et spécialiste en littérature, avec une expertise des journaux historiques belges en langue française, des dessins humoristiques qui y apparaissent  et de l'annotation des entités nommées."
template_0=f"{personality}"


In [45]:
def llm_output(content_user,model_1=model,content_sys=template_0,basemodel_class = NER):
    
    llm = ChatTogether(
      model=model_1,
      temperature=0,
      api_key=TOGETHER_API_KEY,
  )
    messages=[
        (
            "system", content_sys,
        ),
        (
            "human",content_user,
        ),
    ]

    response = llm.invoke(messages)
    print("RESPONSE CONTENT:\n", response.content)
    test_result = basemodel_class.model_validate(from_json(response.content, allow_partial = True))
    result = test_result.model_dump()
    
    return result

In [34]:
def text_splitter(sample_text, chunk_size = 64000):
  custom_text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap  = 20,
      length_function = len,

  )
  texts = custom_text_splitter.create_documents([sample_text])
  texts_content = [text.page_content for text in texts]

  return texts_content

In [35]:
schema_entity={"personne": ["string"],
        "lieu": ["string"],
        "organisation": ["string"]
      }

In [36]:
categories = """
personne : noms propres de personnes,
lieu : noms propres de lieux,
organisation : noms propres d’organisations"""

In [37]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKC", text)  
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)  
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()  
    text = re.sub(r"(\w)([.,;!?])", r"\1 \2", text)  
    return text

In [38]:
input_folder = Path(r"C:\Users\bauke\OneDrive - KU Leuven\Documents\Documenten\5 digital humanities\stage\articles-verbetering")
output_data = []

data = ({"filename": [], "text": []})

for file_path in input_folder.glob("*.txt"):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    
    data["filename"].append(file_path.name)
    data["text"].append(text)

df = pd.DataFrame(data)
        
df["cleaned_text"] = df["text"].apply(preprocess_text)

In [39]:
df["chunks"] = df.cleaned_text.apply(text_splitter)
df = df.explode("chunks")

In [47]:
chunk_size=20000
templates_text=[]
for text in df["chunks"]:
  chunks=text_splitter(text, chunk_size)
  templates=[]
  for sentence in chunks:
  
    template_1 = f"""
          Tu es un expert des journaux historiques belges en français. 
          Ta tâche est d'extraire les entités nommées pertinentes de la phrase donnée selon les catégories suivantes :
          {categories}

          Réponds uniquement en format JSON, sans ajouter de commentair et n’incluez pas de mise en forme Markdown comme les triples accents graves (```).
          Le format JSON attendu est le suivant :
          {schema_entity}

          Voici quelques exemples pour t’inspirer :

          Exemple 1 :
          Phrase : <<<Le roi Léopold II a visité Bruxelles en 1895.>>>

          Étape 1 : Identifier les entités nommées potentielles
          - roi Léopold II
          - Bruxelles

          Étape 2 : Catégoriser chaque entité
          - roi Léopold II : personne
          - Bruxelles : lieu

          Étape 3 : Formater la réponse

          {{
            "personne": ["roi Léopold II"],
            "lieu": ["Bruxelles"],
            "organisation": []
          }}
          Exemple 1 :
          Phrase : <<<Marcel Antoine , président fondateur du groupement « La Mine souriante ». est décédé inopinément à son domicile , à Bruxelles .>>>

          Étape 1 : Identifier les entités nommées potentielles
          - Marcel Antoine
          - La Mine souriante
          - Bruxelles

          Étape 2 : Catégoriser chaque entité
          - Marcel Antoine : Personne
          - La Mine souriante : organisation
          - Bruxelles : lieu

          Étape 3 : Formater la réponse

          {{
            "personne": ["Marcel Antoine"],
            "lieu": ["Bruxelles"],
            "organisation": ["La Mine souriante]
          }}

          Exemple 1 :
          Phrase : <<<"13 h . 30 RADIO-SCHAERBEEK : « Slache-Magazine » de Marcel Antoine">>>

          Étape 1 : Identifier les entités nommées potentielles
          - Marcel Antoine
          - RADIO-SCHAERBEEK
          

          Étape 2 : Catégoriser chaque entité
          - Marcel Antoine : Personne
          - RADIO-SCHAERBEEK : organisation

          Étape 3 : Formater la réponse

          {{
            "personne": ["Marcel Antoine"],
            "lieu": [],
            "organisation": ["RADIO-SCHAERBEEK"]
          }}


          Maintenant, n’utilise pas les exemples ci-dessus, ils ne servent que de référence.

          Question : {question}
          Phrase à analyser : <<<{sentence}>>>

          Réponse :
        """
    templates.append(template_1)
  templates_text.append(templates)

In [54]:
json_output = {}

for index, (templates, row) in enumerate(zip(templates_text, df.to_dict(orient="records"))):
    ner_result = {
        "personne": [],
        "lieu": [],
        "organisation": []
    }

    for sub_template in templates:
        sub_result = llm_output(sub_template, model, template_0)

        for key in ner_result.keys():
            if key in sub_result and isinstance(sub_result[key], list):
                ner_result[key] += sub_result[key]

    for key in ner_result:
        ner_result[key] = list(set(ner_result[key]))

    json_output[row["filename"]] = {
        "text": row["text"],
        "cleaned_text": row["cleaned_text"],
        "people": ner_result["personne"],
        "locations": ner_result["lieu"],
        "organisations": ner_result["organisation"]
    }

llama_results = "llama_results3.json"
with open(llama_results, "w", encoding="utf-8") as f:
    json.dump(json_output, f, indent=4, ensure_ascii=False)

import pandas as pd
df_output = pd.DataFrame.from_dict(json_output, orient="index").reset_index(names="filename")


RESPONSE CONTENT:
 {
  "personne": ["Marcel Antoine", "E . Delhaes"],
  "lieu": [],
  "organisation": []
}
RESPONSE CONTENT:
 {
  "personne": ["Marcel Antoine", "Georges Villiers", "Paginrani", "Pills", "Tabet", "Maurice Gilman", "Pagllarani", "Rsrt"],
  "lieu": ["Spa", "Aranjuez", "Bruxelles"],
  "organisation": ["Radio-Schaerbeek-Bruxelles", "Opêra-Gomlque", "Opéra-Comique"]
}
RESPONSE CONTENT:
 {
  "personne": ["Marcel Antoine"],
  "lieu": ["Bruxelles"],
  "organisation": ["Radio-Schaerbeek-Bruxelles", "Salon de la mine souriante"]
}
RESPONSE CONTENT:
 {
  "personne": ["Marcel Antoine", "Adolphe Mar", "Reginald Denny", "Jean-Pierre Aumont", "Paul Reboux", "Abel Hermant", "Maurice de Waleffe", "René CHASSART", "Ritche"],
  "lieu": ["Bruxelles", "Mon Village", "Touraine"],
  "organisation": ["La Mine Souriante", "Radio qui rit"]
}
RESPONSE CONTENT:
 {
  "personne": [],
  "lieu": ["Bruxelles"],
  "organisation": ["LES SUN'KIST'S", "Radio-Schaerbeek"]
}
RESPONSE CONTENT:
 {
  "personne"

In [ ]:
df_output.head()

In [55]:
flat_json_output = [
    {
        "filename": key,
        "text": value["text"],
        "cleaned_text": value["cleaned_text"],
        "people": ", ".join(set(value["people"])), 
        "locations": ", ".join(set(value["locations"])),
        "organisations": ", ".join(set(value["organisations"]))  
    }
    for key, value in json_output.items()
]
import json
output_file2 = "flat_llama_results.json"
with open(output_file2, "w", encoding="utf-8") as f:
    json.dump(flat_json_output, f, indent=4, ensure_ascii=False)

print(f"flat NER results saved to {output_file2}")